In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import ast
from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

In [2]:
df = pd.read_csv('111.csv')

# mask =(df.groupby('antigen_label')['antigen_label'].transform('count') == 1)

In [3]:
# df = df[~mask]
df = df.drop_duplicates()
encoder = LabelEncoder()
label_rencoded = encoder.fit_transform(df['antigen_label'].tolist())
df['label_rencoded'] = label_rencoded
df

,vdjdb.score,mhc.b_label,antigen_label,crd3_encode,v.seg_0,v.seg_1,v.seg_2,v.seg_3,v.seg_4,v.seg_5,...,j.segm_2,j.segm_3,mhc.a_0,mhc.a_1,mhc.a_2,mhc.a_3,mhc.a_4,mhc.a_5,mhc.a_6,label_rencoded
0,1,0,66,"[0.5393931865692139, 2.0628302097320557, 0.411...",0,1,0,0,1,1,...,0.0,1.0,0,0,0,0,0,0,1,0
1,1,0,66,"[-0.8504062294960022, 0.17268569767475128, 0.2...",1,1,0,1,1,1,...,1.0,0.0,0,0,0,0,0,0,1,0
2,1,0,66,"[-0.534170925617218, 0.2944409251213074, 0.852...",0,1,0,0,1,0,...,0.0,1.0,0,0,0,0,0,0,1,0
7,1,0,66,"[-1.7072793245315552, 0.28665441274642944, -1....",0,0,0,0,1,1,...,0.0,1.0,0,0,0,0,0,0,1,0
11,1,0,66,"[-0.30603480339050293, -0.16719016432762146, -...",0,1,0,1,1,1,...,0.0,0.0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3936,3,34,229,"[-0.534170925617218, 0.2944409251213074, 0.852...",0,1,0,0,1,0,...,0.0,0.0,1,1,0,0,1,0,1,5
3937,1,24,668,"[-0.534170925617218, 0.2944409251213074, 0.852...",0,1,1,0,0,0,...,0.0,1.0,1,1,0,0,1,0,1,14
3938,3,22,668,"[-0.534170925617218, 0.2944409251213074, 0.852...",0,1,0,0,0,0,...,1.0,0.0,1,1,0,0,1,1,1,14
3940,3,25,668,"[-0.534170925617218, 0.2944409251213074, 0.852...",0,1,1,0,0,0,...,1.0,0.0,1,1,0,0,1,1,1,14


In [4]:
X = df[['crd3_encode', 'v.seg_0', 'v.seg_1', 'v.seg_2', 'v.seg_3', 'v.seg_4', 'v.seg_5','vdjdb.score',
              'j.segm_0', 'j.segm_1', 'j.segm_2', 'j.segm_3',
              'mhc.a_0', 'mhc.a_1', 'mhc.a_2', 'mhc.a_3', 'mhc.a_4', 'mhc.a_5', 'mhc.a_6', 'mhc.b_label']].fillna(0)
y = df['label_rencoded']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train)

In [44]:
y_train

array([ 2,  4, 19, ...,  9, 17, 18], dtype=int64)

In [22]:
X_train['vdjdb.score'].values

array([1, 3, 1, ..., 1, 1, 2], dtype=int64)

In [7]:
# X_train_copy_list = []
# y_train_copy_list = []

# # 遍历 X_train 和 y_train 中的每一行
# for index, row in X_train.iterrows():
#     # 根据 score 列的值确定要复制的数量
#     num_copies = row['vdjdb.score']
#     # 复制当前行，并添加到列表中
#     if num_copies > 0:
#         X_train_copy_list.extend([row] * num_copies)
#         y_train_copy_list.extend([y_train[index]] * num_copies)

# # 将列表中的数据片段合并为新的 DataFrame
# X_train_copy = pd.concat(X_train_copy_list, axis=1).T
# y_train_copy = pd.Series(y_train_copy_list, name='label_rencoded')

# X_train_copy.reset_index(drop=True, inplace=True)
# y_train_copy.reset_index(drop=True, inplace=True)

# X_train_new = pd.concat([X_train, X_train_copy], ignore_index=True)
# y_train_new = pd.concat([y_train, y_train_copy], ignore_index=True)

# # 重新设置索引
# X_train_new.reset_index(drop=True, inplace=True)
# y_train_new.reset_index(drop=True, inplace=True)

In [40]:
y_train

array([ 2,  4, 19, ...,  9, 17, 18], dtype=int64)

In [6]:
def dimensionality_reduction(df, n_components1, n_components2):
    vdj = df[['v.seg_0', 'v.seg_1', 'v.seg_2', 'v.seg_3', 'v.seg_4', 'v.seg_5',
              'j.segm_0', 'j.segm_1', 'j.segm_2', 'j.segm_3']]
    mhc = df[['mhc.a_0', 'mhc.a_1', 'mhc.a_2', 'mhc.a_3', 'mhc.a_4', 'mhc.a_5', 'mhc.a_6', 'mhc.b_label']]

    vdj_lists = vdj.values.tolist()
    mhc_lists = mhc.values.tolist()
    
    cdr3_kmer = list(df['crd3_encode'].apply(lambda x: ast.literal_eval(x)))
    
    
    pca = PCA(n_components = n_components1)
    pca_data = pca.fit_transform(cdr3_kmer)
    
    tcr_list = []
    for cdr3, vdj, mhc in zip(pca_data, vdj_lists, mhc_lists):
        tcr_list.append(list(cdr3) + vdj + mhc)
    
    tsne = manifold.TSNE(n_components=n_components2, init='pca', random_state=1, learning_rate=100)

    reduced_data = tsne.fit_transform(tcr_list)
    return reduced_data

In [7]:
X_train_preprocessing = dimensionality_reduction(X_train, 50, 3)
X_test_preprocessing = dimensionality_reduction(X_test, 50, 3)

In [98]:
X_train_preprocessing

array([[-2.14240406e+00, -1.44890469e+00,  3.64363774e+00, ...,
         3.78468145e-01,  1.41440749e-01, -7.36705958e-02],
       [-6.13351467e+00,  7.75853136e+00,  6.35270214e+00, ...,
        -8.55259389e-03,  5.84495116e-01,  1.63711027e+00],
       [-4.05254850e+00,  3.52268631e+00, -1.15951863e+00, ...,
         1.29464021e+00, -1.33194166e+00, -7.00853422e-01],
       ...,
       [ 1.66463834e+01,  4.36984612e+00, -5.84662827e+00, ...,
        -1.99694394e-01, -1.80346233e+00,  2.53444470e+00],
       [-7.34403417e-01, -6.07679983e+00,  7.59871323e+00, ...,
        -3.32281379e+00, -4.43176297e+00, -2.79983164e-01],
       [ 2.32816578e+00, -1.22941641e+01,  5.60778483e+00, ...,
        -3.03898055e-01,  1.62816633e+00,  1.12846648e+00]])

In [59]:
np.unique(y_train)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20], dtype=int64)

In [10]:
def custom_scorer(y_true, y_pred):
    auc_scores = []
    for i in range (len(np.unique(y_true))):
        y_pred_class = y_pred[:, i]
        y_true_class = (y_true == i)
        auc_scores.append(roc_auc_score(y_true_class, y_pred_class))
    auc_score = sum(auc_scores) / len(np.unique(y_train))
    return auc_score

custom_scorer = make_scorer(custom_scorer)

In [25]:
# Define the parameter grid
param_grid = {
    'n_estimators': [80, 100, 150, 200],
    'learning_rate': [0.02, 0.05, 0.1],
    'max_depth': [6, 8, 10, 12],
    'min_child_weight': [2, 3, 4, 5, 6],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha' : [0, 0.05, 0.1, 1]
}

# Create an XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(np.unique(y_train)), random_state = 42, nthread = 2, eval_metric = 'mlogloss')

# Perform grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose = 1, scoring='roc_auc_ovr')
grid_search.fit(X_train_preprocessing, y_train, sample_weight=X_train['vdjdb.score'].values)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 15360 candidates, totalling 76800 fits
Best parameters:  {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 12, 'min_child_weight': 2, 'n_estimators': 150, 'reg_alpha': 0.05, 'subsample': 0.7}
Best score:  0.42608767358247845


In [29]:
# Import the necessary modules
from xgboost import XGBClassifier

# Create an instance of XGBClassifier
xgb_model = XGBClassifier(objective='multi:softprob', num_class= len(np.unique(y_train)), 
          learning_rate=0.1, n_estimators= 150, max_depth= 12, min_child_weight= 2, seed=0, gamma= 0, reg_alpha = 0.05)

# Train the model
xgb_model.fit(X_train_preprocessing, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              n_estimators=150, n_jobs=None, num_class=21,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [13]:
# Create an instance of XGBClassifier
xgb_model = XGBClassifier(objective='multi:softprob', num_class= len(np.unique(y_train)), 
          learning_rate=0.1, n_estimators= 10)

# Train the model
xgb_model.fit(X_train_preprocessing, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=None, num_class=21,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [14]:
pre_y_test = xgb_model.predict_proba(X_test_preprocessing)

In [15]:
# 计算每个类别的AUC
auc_scores = []
for i in range (len(np.unique(y_test))):
    y_pred_class = pre_y_test[:, i]  # 提取第i个类别的预测概率
    y_true_class = (y_test == i)  # 将真实标签转化为二分类问题
    auc_scores.append(roc_auc_score(y_true_class, y_pred_class))

# 计算多分类任务的AUC指标
mean_auc = sum(auc_scores) / len(np.unique(y_train))

print('AUC: ', mean_auc)

AUC:  0.5957571199071903


In [54]:
def GridSearch(cv_params, other_params, X_train_preprocessing, y_train_new):
    model = xgb.XGBRegressor(**other_params)
    optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4)
    optimized_GBM.fit(X_train_preprocessing, y_train_new)
    print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
    print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

In [55]:
cv_params = {'n_estimators': [10, 30, 55, 100, 150, 200]}
other_params = {'objective': 'multi:softmax', 'num_class': y_train_new.nunique(), 'learning_rate': 0.1, 'n_estimators': 500,
                'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

GridSearch(cv_params, other_params, X_train_preprocessing, y_train_new)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
参数的最佳取值：{'n_estimators': 100}
最佳模型得分:0.9534223154377157


In [60]:
cv_params = {'max_depth': [20, 50, 100, 150, 200], 'min_child_weight': [1, 2, 3, 4, 5, 6]}
other_params = {'objective': 'multi:softmax', 'num_class': y_train_new.nunique(), 
                'learning_rate': 0.1, 'n_estimators': 150, 'max_depth': 20, 'min_child_weight': 6, 'seed': 0,
                'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
GridSearch(cv_params, other_params, X_train_preprocessing, y_train_new)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
参数的最佳取值：{'max_depth': 20, 'min_child_weight': 6}
最佳模型得分:0.9613372725962229


In [61]:
cv_params = {'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]}
other_params = {'objective': 'multi:softmax', 'num_class': y_train_new.nunique(), 
                'learning_rate': 0.1, 'n_estimators': 150, 'max_depth': 20, 'min_child_weight': 6, 'seed': 0,
                'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
GridSearch(cv_params, other_params, X_train_preprocessing, y_train_new)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
参数的最佳取值：{'gamma': 0.1}
最佳模型得分:0.9613372725962229


In [24]:
cv_params = {'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]}
other_params = {'objective': 'multi:softmax', 'num_class': y_train_new.nunique(), 
                'learning_rate': 0.1, 'n_estimators': 150, 'max_depth': 20, 'min_child_weight': 1, 'seed': 0,
                'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.1, 'reg_alpha': 0, 'reg_lambda': 1}
GridSearch(cv_params, other_params, X_train_preprocessing, y_train_new)

NameError: name 'y_train_new' is not defined

In [23]:
params = {'objective': 'multi:softmax', 'num_class': y_train.nunique(), 
          'learning_rate': 0.05, 'n_estimators': 150, 'max_depth': 100, 'min_child_weight': 1, 'seed': 0,
          'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.1, 'reg_alpha': 0, 'reg_lambda': 1}
model = xgb.XGBRegressor(**other_params)
model.fit(X_train_preprocessing, y_train)

NameError: name 'other_params' is not defined

In [64]:
X_test_preprocessing = dimensionality_reduction(X_test, 50, 3)

In [71]:
# dtest = xgb.DMatrix(X_test_preprocessing)
pre_y_test = model.predict(X_test_preprocessing)

In [72]:
pre_y_test

array([ 3.,  6., 12., 17.,  8., 18.,  5., 15., 13., 12., 13.,  6., 15.,
        2., 13., 13., 13.,  4.,  9.,  9., 13., 18., 18., 13., 11.,  4.,
       13., 18.,  6.,  8., 18.,  6.,  9., 13.,  7., 13., 13., 18., 16.,
       18., 18., 17.,  9., 18.,  0., 13., 13.,  6., 13., 16., 13.,  5.,
       10., 15., 18., 10., 13.,  9.,  9., 13.,  5.,  3.,  4.,  2.,  5.,
        7.,  5., 13.,  9., 13., 19.,  0., 13.,  9., 15., 15., 11., 13.,
       13.,  6., 13., 13., 13., 13., 13., 18.,  0., 13., 14., 11., 18.,
        6.,  9., 13.,  3.,  8.,  6., 13., 17.,  7., 13., 13.,  7., 17.,
        6., 14.,  6.,  6., 18.,  0., 11.,  3., 11.,  9.,  7.,  2., 13.,
       13., 15., 17., 10.,  0., 11.,  0., 15.,  6., 18.,  5., 15., 15.,
        4.,  6., 11.,  0., 10., 13.,  8.,  2., 13., 13., 13., 13., 13.,
        6.,  3.,  7., 13.,  6., 12., 13.,  3., 18.,  9., 14., 11., 13.,
       13.,  7., 13., 13.,  3., 13., 13.,  6.,  0., 13., 15., 12., 13.,
       18.,  9.,  8., 13., 15.,  7., 15., 15., 11., 15., 11., 10

In [70]:
from sklearn.metrics import roc_auc_score

0.7036639429409204


d:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
